In [ ]:
#Cell_1.
import openai
import re
import os
from openai import OpenAI
from openai.types.chat import ChatCompletionMessageParam
from mylocalenv import OPENAI_API_KEY

oai = OpenAI(
    api_key = OPENAI_API_KEY,
    base_url = "https://api.openai.iniad.org/api/v1"
)

In [ ]:
#Cell_2.
with open('world_country_population.csv', 'r', encoding='utf-8') as file:
    file_content = file.read()

In [ ]:
#Cell_3.
from typing import List
messages: List[ChatCompletionMessageParam] =[
       {"role": "system", "content": "あなたはデータサイエンティストです。"},
       {"role": "user", "content": "いまアップロードしたデータ'world_country_population.csv'について解説してください。"},
       {"role": "user", "content": file_content}
]

resp = oai.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
)

print(resp.choices[0].message.content)

In [ ]:
#Cell_4.
messages.append({"role": "assistant", "content": resp.choices[0].message.content})
messages.append({"role": "user", "content": "このworld_country_population.csvから2000年時点の中国、インド、日本の人口を取得して、世界地図上に色付けしてください。"})

resp = oai.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
)

print(resp.choices[0].message.content)

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# データを読み込む
data = pd.read_csv('world_country_population.csv')

# 中国、インド、日本の2000年の人口を取得
population_data = data[data['Country Name'].isin(['China', 'India', 'Japan'])]
population_data = population_data[['Country Name', 'Country Code', '2000']]

# 各国の2000年の人口を取得
population_data['2000'] = population_data['2000'].astype(float)

print(gpd.__version__)

# 世界地図を読み込む
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# 人口データを世界地図にマージする
world = world.merge(population_data, how='left', left_on='iso_a3', right_on='Country Code')

# マップを描画する
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world.boundary.plot(ax=ax, linewidth=1)
world[world['Country Name'].notnull()].plot(column='2000', ax=ax, legend=True,
                                            legend_kwds={'label': "Population in 2000",
                                                         'orientation': "horizontal"},
                                            cmap='OrRd',
                                            missing_kwds={"color": "lightgrey",
                                                          "edgecolor": "black",
                                                          "hatch": "///",
                                                          "label": "Missing data"})

# タイトル
plt.title('Population of China, India, and Japan in 2000')
plt.show()


# skiprowsを追加する必要があった
# GeoPandasはインストール時バージョン1未満を指定する必要がある

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# データを読み込む
data = pd.read_csv('world_country_population.csv', skiprows=4)

# 中国、インド、日本の2000年の人口を取得
population_data = data[data['Country Name'].isin(['China', 'India', 'Japan'])]
population_data = population_data[['Country Name', 'Country Code', '2000']]

# 各国の2000年の人口を取得
population_data['2000'] = population_data['2000'].astype(float)
print(population_data["2000"])
print(gpd.__version__)

# 世界地図を読み込む
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# 人口データを世界地図にマージする
world = world.merge(population_data, how='left', left_on='iso_a3', right_on='Country Code')

# マップを描画する
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world.boundary.plot(ax=ax, linewidth=1)
world[world['Country Name'].notnull()].plot(column='2000', ax=ax, legend=True,
                                            legend_kwds={'label': "Population in 2000",
                                                         'orientation': "horizontal"},
                                            cmap='OrRd',
                                            missing_kwds={"color": "lightgrey",
                                                          "edgecolor": "black",
                                                          "hatch": "///",
                                                          "label": "Missing data"})

# タイトル
plt.title('Population of China, India, and Japan in 2000')
plt.show()


日本がぬられてない